# Terminate Game Puzzle

This is a puzzle output `CREATE_COIN` conditions according to the `play_result`. Also, if the tic tac toe coin puzzle is wrapped inside the singleton top layer, the `IS_SINGLETON` is set, and terminate conditions will include `CREATE_COIN ... -113` to terminate the singleton coin.

```lisp
(mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
    (include condition_codes.clib)

    (defun p2-players (PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
       (if (= -1 play_result)
            ; tie game
            (list
                (list CREATE_COIN PLAYER_ONE_HASH P2_AMOUNT)
                (list CREATE_COIN PLAYER_TWO_HASH P2_AMOUNT)
            )
            ; there is a winner
            (list
                (list CREATE_COIN
                    (if (= 'x' play_result) PLAYER_ONE_HASH PLAYER_TWO_HASH)
                    (* P2_AMOUNT 2) 
                )
            )
        )
    )

    (if IS_SINGLETON
        (c
            (list CREATE_COIN PLAYER_ONE_HASH -113) ; destroy a singleton coin
            (p2-players PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
        )
        (p2-players PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
    )
)
```

In [1]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)
from chia.types.blockchain_format.program import Program

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle)

sys.path.insert(0, "./code")
import tic_tac_toe

terminate_puzzle = load_program("./code/terminate-game.clsp", ["./code", "../../../shared"])

## Terminate Singleton Coin

In [2]:
# (mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
puzzle = terminate_puzzle.curry(1, '0xdeadbeef' , '0xcafef00d', 100)
print_program(puzzle.run(Program.to([-1])))
print_program(puzzle.run(Program.to(['x'])))
print_program(puzzle.run(Program.to(['o'])))

((51 "0xdeadbeef" -113) (51 "0xdeadbeef" 100) (51 "0xcafef00d" 100))
((51 "0xdeadbeef" -113) (51 "0xdeadbeef" 200))
((51 "0xdeadbeef" -113) (51 "0xcafef00d" 200))


## Terminate Normal Coin

In [3]:
# (mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
puzzle = terminate_puzzle.curry(0, '0xdeadbeef' , '0xcafef00d', 100)
print_program(puzzle.run(Program.to([-1])))
print_program(puzzle.run(Program.to(['x'])))
print_program(puzzle.run(Program.to(['o'])))

((51 "0xdeadbeef" 100) (51 "0xcafef00d" 100))
((51 "0xdeadbeef" 200))
((51 "0xcafef00d" 200))
